# Segmenting and Clustering Neighborhoods in Toronto

**1. Scraping the Wikipedia page https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M and transform it as a pandas DataFrame**

*mainly based on http://srome.github.io/Parsing-HTML-Tables-in-Python-with-BeautifulSoup-and-pandas/*

In [51]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
    
class HTMLTableParser:
    def parse_url(self, url):
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'lxml')
        return(self.parse_html_table(soup.find_all('table')[0]))
    
    def parse_html_table(self, table):
        n_columns = 0
        n_rows=0
        column_names = []
    
        # Find number of rows and columns
        # we also find the column titles if we can
        for row in table.find_all('tr'):
            # Determine the number of rows in the table
            td_tags = row.find_all('td')
            if len(td_tags) > 0:
                n_rows+=1
                if n_columns == 0:
                    # Set the number of columns for our table
                    n_columns = len(td_tags)
                    
            # Handle column names if we find them
            th_tags = row.find_all('th') 
            if len(th_tags) > 0 and len(column_names) == 0:
                for th in th_tags:
                    column_names.append(th.get_text().replace('\n','').replace(' ', ''))
    
        # Safeguard on Column Titles
        if len(column_names) > 0 and len(column_names) != n_columns:
            raise Exception("Column titles do not match the number of columns")
    
        columns = column_names if len(column_names) > 0 else range(0,n_columns)
        df = pd.DataFrame(columns = columns, index= range(0,n_rows))
        row_marker = 0
        for row in table.find_all('tr'):
            column_marker = 0
            columns = row.find_all('td')
            for column in columns:
                df.iat[row_marker,column_marker] = column.get_text().replace('\n','')
                column_marker += 1
            if len(columns) > 0:
                row_marker += 1

            
        return df

In [52]:
hp = HTMLTableParser()
df = hp.parse_url("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M") # Grabbing the table from the tuple
df

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


**2. process the cells that have an assigned borough.** 

Ignore cells with a borough that is *Not assigned*.

In [53]:
df = df[df.Borough != "Not assigned"]
df

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


**3. More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11  in the above table.**

In [54]:
df[df.PostalCode == 'M5A']
# already done

,PostalCode,Borough,Neighbourhood
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


**4. If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.**

In [55]:
df[df.Neighbourhood == 'Not assigned']
# after step 2, no Neighbourhood is "Not assigned" anymore. No process to do

,PostalCode,Borough,Neighbourhood


 **5. use the .shape method to print the number of rows of your dataframe.**

In [56]:
df.shape

(103, 3)